# Scraping Twitter

Twitter constitue une source de données utiles pour des projets de recherches en sciences humaines et sociales. Que ce soit pour de l'analyse de réseau ou de l'analyse de contenu, il existe différents outils permettant de récupérer efficacement des données sur ce réseau social.

## Utiliser l'API de Twitter

On peut retrouver un guide pour l'API de Twitter à cette adresse : https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/introduction. Pour les références de l'API de Twitter, on consultera cette adresse : https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/api-reference.

### Authentification

La première chose à faire avant d'explorer l'API de Twitter plus avant est de renseigner toutes les informations pour se connecter à l'API de Twitter. Cela nécessite au préalable la création :
* D'un compte Twitter personnel
* D'un compte développeur Twitter
* La création d'un projet enregistré dans son profil développeur Twitter

In [158]:
import json

with open("twitter_secret.json", "r") as twitter_account:
    twitter_secret = json.load(twitter_account)

Une fois que tous les tokens d'accès sont récupérés, il s'agit de se connecter à l'API en passant toutes les autorisations.

In [159]:
consumer_key = twitter_secret["consumer_key"]
consumer_secret = twitter_secret["consumer_secret"]
bearer_token = twitter_secret["bearer_token"]
auth_token = twitter_secret["auth_token"]
auth_secret = twitter_secret["auth_secret"]

Il existe différentes méthodes d'authentification sur l'API Twitter :
* **OAuth 1.0a :** il s'agit de la méthode la plus classique qui consiste à passer les consumer_key et consumer_secret (qui représentent l'application) en paramètre de chaque requête faite à l'API Twitter, accompagné du token_key et du token_secret (qui représentent l'utilisateur à travers lequel l'application va envoyer les requêtes)
* **OAuth 2.0 Bearer Token :** cette méthode est plutôt utilisée lorsqu'on souhaite connecter une application à Twitter qui a uniquement besoin de lire des données (il est possible de générer un Bearer Token à la volée en envoyant le consumer_key et le consumer_secret au chemin de l'API Twitter oauth2/token avec une méthode POST)

### Effectuer une recherche

Maintenant que nous disposons de toutes les informations pour se connecter, nous pouvons envoyer une requête HTTP à l'API de Twitter en passant en paramètre les tokens nécessaires.

Commençons par définir quelques fonctions de base qui permettent d'effectuer rapidement une recherche de tweets.

In [199]:
import requests
import json

api_url = "https://api.twitter.com/2/tweets/search/all"
params = {'query': 'pirates'}

def bearer_token_headers(bearer_token):
    headers = {"Authorization": f"Bearer {bearer_token}"}
    return headers    

def connect_endpoint(url, headers, params):
    r = requests.get(url=url, headers=headers, params=params)
    if r.status_code != 200:
        raise Exception(r.status_code, r.text)
    else:
        print(r.status_code)
    return r.json()

In [200]:
headers = bearer_token_headers(bearer_token)
connect_endpoint(url=api_url, headers=headers, params=params)

Exception: (403, '{"client_id":"20023224","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Standard Basic","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}')

## Using Tweepy

Tweepy est un package Python permettant d'accéder facilement aux options de l'API Twitter.

On peut retrouver les références de l'API Tweepy à cette adresse : https://docs.tweepy.org/en/latest/api.html#api-reference.

In [2]:
import tweepy

### Authentification

La première chose à faire avant d'explorer plus avant l'API Tweepy est de renseigner toutes les informations pour se connecter à l'API Twitter. Cela nécessite au préalable la création :
* D'un compte Twitter personnel
* D'un compte développeur Twitter
* La création d'un projet enregistré dans son profil développeur Twitter

In [148]:
import json

with open("twitter_secret.json", "r") as twitter_account:
    twitter_secret = json.load(twitter_account)

Une fois que tous les tokens d'accès sont récupérés, il s'agit de se connecter à l'API en passant toutes les autorisations.

In [149]:
consumer_key = twitter_secret["consumer_key"]
consumer_secret = twitter_secret["consumer_secret"]
bearer_token = twitter_secret["bearer_token"]
auth_token = twitter_secret["auth_token"]
auth_secret = twitter_secret["auth_secret"]

Maintenant que toutes les informations de connexion sont stockées dans des variables, on peut directement les réutiliser pour se connecter à l'API Twitter.

In [150]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(auth_token, auth_secret)

api = tweepy.API(auth)

### Récupérer des tweets

Une fois que l'on s'est connecté, on peut utiliser les méthodes de recherche pour récupérer nos premiers Tweets.

In [111]:
timeline_tweets = api.home_timeline()
# On récupère seulement les cinq premiers tweets
for tweets in timeline_tweets[:5]:
    print(tweets.text)

Bon je crois que je vais arrêter de poster mes graphiques pourris auxquels personne ne bite rien pour les remplacer… https://t.co/kKLii28eKA
Et même sur l'évolution du nombre de lits de réanimation occupés pour Covid19 on a un peu de vert - qui l'eût cru? https://t.co/TeR2EFThpa
Un peu de vert aussi pour l'évolution du nombre de lits occupés à l'hôpital pour Covid19 - ça ne mange pas de pain. https://t.co/AapibyVLl5
Cela pourrait être pertinent de superposer cette carte avec celle des variants anglais et sud-africains. https://t.co/rOjr87jXkR
La carte du jour : ça fait du bien de voir un peu de vert de temps en temps - je commençais à me lasser du rouge!… https://t.co/nvcyYhezLV


On peut aussi parcourir Twitter en récupérant le nom d'un utilisateur.

In [109]:
twitter_user = api.get_user("EmmanuelMacron")
# Récupérons les 5 premiers followers d'EmmanuelMacron
for follower in twitter_user.followers()[:5]:
    print(follower.screen_name)

MuanguR
lechatquiment
Virgile1123
AXEL71725540
Camille10042019


### Utiliser le curseur

L'API Twitter renvoie beaucoup d'informations qui nécessitent de les parcourir par pages.

Pour simplifier ces opérations, **Tweepy** dispose d'un objet `Cursor`. Il gère par lui-même les appels aux pages suivantes de telle sorte qu'il est beaucoup plus facile pour l'utilisateur de parcourir les pages en utilisant une seule et simple boucle.

In [121]:
for status in tweepy.Cursor(api.user_timeline, screen_name="EmmanuelMacron").items(5):
    print(status.text)

C’est par notre vigilance collective que nous réussirons.
Tester, au moindre symptôme.
Alerter, pour limiter la pro… https://t.co/2uMax2Vovx
Ce qui nous contraint aujourd’hui dans la campagne de vaccination, c’est l’accès aux doses. Nous avons un calendrie… https://t.co/4D5NQih7lg
D'ici à la fin de l'été, nous aurons proposé à tous les Français adultes qui le souhaitent un vaccin.
Nous ne pourrons vaincre le virus que si dans tous les pays d'Europe où nous circulons librement nous arrivons à av… https://t.co/Z8ZkZwDH2o
J'ai confiance en notre capacité à nous mobiliser pour freiner le virus. https://t.co/6dthNeKF0l


### Etendre les contenus de Tweets

Par défaut, l'API Twitter accessible via **Tweepy** retourne le contenu des tweets tronqués à 140 caractères. Cependant, il est possible de passer en paramètre de tout objet `Status` renvoyé par **Tweepy**, le paramètre `tweet_mode`. Celui-ci prend deux valeurs :
* `compat` : limite le nombre de caractères à 140
* `extended` : ne limite pas le nombre de caractères

Pour accéder au texte intégral du tweet, on utilisera alors la propriété `.full_text` et non plus `.text`.

In [122]:
for status in tweepy.Cursor(api.user_timeline, screen_name="EmmanuelMacron", tweet_mode="extended").items(5):
    print(status.full_text)

C’est par notre vigilance collective que nous réussirons.
Tester, au moindre symptôme.
Alerter, pour limiter la propagation.
Protéger, en respectant l’isolement.
Ce qui nous contraint aujourd’hui dans la campagne de vaccination, c’est l’accès aux doses. Nous avons un calendrier. Entre fin février et début mars, 4 sites français vont produire le vaccin. Notre mobilisation reste totale.
D'ici à la fin de l'été, nous aurons proposé à tous les Français adultes qui le souhaitent un vaccin.
Nous ne pourrons vaincre le virus que si dans tous les pays d'Europe où nous circulons librement nous arrivons à avoir la même campagne de vaccination. Il nous faut vacciner en Européens. C'est notre stratégie.
J'ai confiance en notre capacité à nous mobiliser pour freiner le virus. https://t.co/6dthNeKF0l
